In [ ]:
import numpy as np
import pandas as pd
import random
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input
import matplotlib.pyplot as plt
from keras.preprocessing import text

In [ ]:
def load_data():
    path_train="/kaggle/input/foursquare-location-matching/train.csv"
    train = pd.read_csv(path_train)
    path_test="/kaggle/input/foursquare-location-matching/test.csv"
    test = pd.read_csv(path_test)
    x_cols=["latitude","longitude"]
    y_cols='point_of_interest'
    x_max=train[["latitude","longitude"]].max().tolist()
    x_min=train[["latitude","longitude"]].min().tolist()
    x_test=test[x_cols].values
    x_train=train[x_cols].values
    y_train=train[y_cols].values
    return x_train,y_train,x_test,x_max,x_min

In [ ]:
def load_pairs():
    path="/kaggle/input/foursquare-location-matching/pairs.csv"
    data = pd.read_csv(path)
    x1_cols=["latitude_1","longitude_1"]
    x2_cols=["latitude_2","longitude_2"]
    y_cols='match'
    x_max=data[x1_cols].max().tolist()
    x_min=data[x1_cols].min().tolist()
    x1=data[x1_cols].values
    x2=data[x2_cols].values
    y=np.multiply(data[y_cols].values, 1)
    return x1,x2,y,x_max,x_min

In [ ]:
x1,x2,y,x_max,x_min=load_pairs()

In [ ]:
x1=np.divide(np.subtract(x1,x_min),np.subtract(x_max,x_min))
x2=np.divide(np.subtract(x2,x_min),np.subtract(x_max,x_min))

In [ ]:
from keras import backend as K
from keras.layers import GRU

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


input = Input((2,))
x = Flatten()(input)
x = Dense(128, activation='relu')(x)
dense = Model(input, x)

input1 = Input((2,))
input2 = Input((2,))

dense1 = dense(input1)
dense2 = dense(input2)

merge_layer = Lambda(euclidean_distance)([dense1,dense2])
dense_layer = Dense(1, activation="sigmoid")(merge_layer)
model = Model(inputs=[input1, input2], outputs=dense_layer)

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
model.fit([x1, x2], y, batch_size=32, epochs=3)